In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

### TensorFlow Computational Graph
* TF internally represents its computation using a data flow graph consisting of:
    * A set of nodes (operations)
    * A set of directed arcs/edges (data on which operations are performed)
    
* `tf.Graph` represents a collection of tf.Operations
* You can create operations by writing out equations.
* By default, there is a graph that you can access with: `tf.get_default_graph()` - and any new operations are added to this graph.
* The result of a tf Operation is a tf Tensor, which holds the values.

<img src="../pics/computation_graph_explained.jpg" width=50%>

### Example Execution Graph for the equation   $\ \ z=d\ x\ c\ =\ (a+b)\ x\ c$

<img src="../pics/simple_execution_graph.jpg" width=50%>

### Example Execution Graph for the series of equations:
$$c\ =\ a\ +\ b$$
$$d\ =\ b\ -\ 1$$
$$e\ =\ c\ x\ d$$

<img src="../pics/simple_computation_graph2.png" width=50%>

### Let's see the above graph in TF code

#### First set up the nodes and edges in the graph

In [ ]:
# Remember we use `placeholder` objects to tell TF to wait for data.
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.add(a,b)
d = tf.subtract(b, 1)
e = tf.multiply(c, d)

#### Now run the graph in a session using a feed_dict to handle input_data

In [ ]:
with tf.Session() as sess:
    a_data, b_data = 3.0, 6.0
    feed_dict = {a: a_data, b: b_data}
    output = sess.run(e, feed_dict=feed_dict)
    print(output)

### Main Components of a TF graph:
* **Variables:** Holds the values for weights and biases between TensorFlow sessions.
* **Tensors:** Sets of values that pass between nodes to perform operations.
* **Placegholders:** Waits for data to come in from the program to the TF graph.
* **Session:** When a session is strarted, TF automatically calcualtes gradients for all of the operations in the graph and uses them. A session is invoked for the purpose of executing the graph.

### Multiple Graphs
* The operation that we created above was automagically added to the graph in TensorFlow. 
* There is a default graph that is instantiated when the TF library is imported. \
* Sometimes, we may want to create our own Graph object instead of using the default graph - for instance when creating multiple models in one file that do not depend on each other.
* Any variables or operations used outside of the `with new_graph.as_default()` context will be added to the default graph that is created when the library is loaded. \
* You can get a handle to the default graph with `tf.get_default_graph()`
* For most cases, it's best to stick to the default graph

In [ ]:
new_graph = tf.Graph()

with new_graph.as_default():
    new_g_const = tf.constant([1., 2.])
    
default_g = tf.get_default_graph()

### Session Objects
* `tf.Session()` is the main TF session object
* `tf.InteractiveSession()` easier to use in Jupyter Notebooks for prototyping
* There is also the new eager execution mode, which does away with the need for Session objects

#### tf.Session()
* Creates an environment in which operations and tensors are evaluated and executed.
* Sessions allocate for their own variables, queus and readers.
* It's important to use the `close()` method when the session is over (or use a context manager).
* Three arguments for a `Session`, all optional.
    1. target - The execution engine to connect to.
    2. graph - The Graph to be launched.
    3. config - A ConfigProto protocol buffer with configuration options for the session.

### Everything needs to be run in a `Session()`

In [ ]:
m1 = tf.constant([[2, 2]])
m2 = tf.constant([[3],
                  [3]])
dot_operation = tf.matmul(m1, m2)

print(dot_operation)  # wrong! no result

In [ ]:
# method1 use session
sess = tf.Session()
result = sess.run(dot_operation)
print(result)
sess.close()

In [ ]:
# method2 use session
with tf.Session() as sess:
    result_ = sess.run(dot_operation)
    print(result_)

In [ ]:
a = tf.constant(1)
b = tf.constant(2)
c = a + b
sess = tf.Session()
print(sess.run(c))
sess.close()

### Running multiple operations in a `Session()`

In [ ]:
x1 = tf.placeholder(dtype=tf.float32, shape=None)
y1 = tf.placeholder(dtype=tf.float32, shape=None)
z1 = x1 + y1

x2 = tf.placeholder(dtype=tf.float32, shape=[2, 1])
y2 = tf.placeholder(dtype=tf.float32, shape=[1, 2])
z2 = tf.matmul(x2, y2)

with tf.Session() as sess:
    # when only one operation to run
    z1_value = sess.run(z1, feed_dict={x1: 1, y1: 2})

    # when run multiple operations
    z1_value, z2_value = sess.run(
        [z1, z2],       # run them together
        feed_dict={
            x1: 1, y1: 2,
            x2: [[2], [2]], y2: [[3, 3]]
        })
    print(z1_value)
    print(z2_value)

#### tf.InteractiveSession()
* Exactly the same as `tf.Session()` but you don't have to explicitly pass the Session object.
* Targeted for use with Jupyter notebooks and allows you to use Tensor.eval() and Operation.run() instead of having to do Session.run() every time you want something computed.
* Will likely be replaced by eager mode in most situations over time, which we will get into later.

In [ ]:
sess = tf.InteractiveSession()
a = tf.constant(1)
b = tf.constant(2)
c = a + b

#print(sess.run(c))
print(c.eval()) # instead of sess.run(c)
sess.close()

### Variables again
* Variables in TensorFlow are managed by the Session. 
* They persist between sessions which are useful because Tensor and Operation objects are immutable. 
* Variables can be created by tf.Variable().
* It is often helpful to name them, so that you can keep track of them in your computation graph.
* Most of the time, you'll want to create these variables as tensors of zeros, ones, or random values, giving the function a shape parameter - e.g. [2, 2, 2] for a 2x2x2 matrix
    * `tf.zeros()` — creates a matrix full of zeros
    * `tf.ones()` — creates a matrix full of ones
    * `tf.random_normal()` — a matrix with random uniform values between an interval
    * `tf.random_uniform()` — random normally distributed numbers
    * `tf.truncated_normal()` — same as random normal but doesn’t include any numbers more than 2 standard deviations.

In [ ]:
# set a variable with an initial value of 1
tensorflow_var = tf.Variable(1, name="my_variable")

# 4x4x4 matrix normally distributed mean 0 std 1
normal = tf.truncated_normal([4, 4, 4], mean=0.0, stddev=1.0)

# setting the above up as a variable
normal_var = tf.Variable(tf.truncated_normal([4,4,4] , mean=0.0, stddev=1.0))

To have these variables initialized you must use TensorFlow’s variable initialization function then pass it to the session. This way when multiple sessions are ran the variables are the same.

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

If you’d like to completely change the value of a variable you can use Variable.assign() operation, this must be run in a session update the value.

In [ ]:
initial_var = tf.Variable(1)
changed_var = initial_var.assign(initial_var + initial_var)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(changed_var)
# 2

In [ ]:
sess.run(changed_var)
# 4

In [ ]:
sess.run(changed_var)
# 8

In [ ]:
sess.close()

#### Counters inside a TF graph
* Sometimes you might want to add a counter inside your model which can be done using the `Variable.assign_add()` method which takes a numeric parameter and increments it by the parameter. 
* Similarly there is `Variable.assign_sub()`.

In [ ]:
counter = tf.Variable(0)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(init)
sess.run(counter.assign_add(1))
# 1

In [ ]:
sess.run(counter.assign_sub(1))
# back to 0

In [ ]:
sess.close()

### Scope
* To control the complexity of models and make them easier to break down into individual pieces TensorFlow has scopes. 
* Scopes are very simple and even help break down your model when using TensorBoard (which will be covered soon). 
* Scopes can even be nested inside of other scopes.

In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()
with tf.name_scope("Scope1"):
    with tf.name_scope("Scope_nested"):
        nested_var = tf.multiply(5, 5)

In [ ]:
tf.get_default_graph().get_operations()

### Variable Scope

In [ ]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])

assert v.name == "foo/bar/v:0"

In [ ]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
    tf.get_variable_scope().reuse_variables()
    v1 = tf.get_variable("v", [1])

assert v1 == v

In [ ]:
tf.reset_default_graph()
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
    
assert v.name == "foo/v:0"

In [ ]:
tf.reset_default_graph()
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    
print(v.name)
print(v1.name)
assert v1 == v

### Name Scope and Variable Scope

In [ ]:
with tf.name_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

In [ ]:
with tf.variable_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

print(v1.name)  # my_scope/var1:0
print(v2.name)  # my_scope/var2:0
print(a.name)   # my_scope/Add:0

In [ ]:
with tf.name_scope("foo"):
    with tf.variable_scope("var_scope"):
        v = tf.get_variable("var", [1])
with tf.name_scope("bar"):
    with tf.variable_scope("var_scope", reuse=True):
        v1 = tf.get_variable("var", [1])
assert v1 == v
print(v.name)   # var_scope/var:0
print(v1.name)  # var_scope/var:0